Trending Topics, Fin
---------------------

Today we close the topic of trends by taking a crack at making our own algorithms. The purpose of this exercise is to have you think through what the concept of trending means, or ought to mean, and then experiment with the data we've been collecting to implement, even a limited version of, your idea. Trending topics have been manipulated to help questionable content go viral, and having a basic understanding of what goes into an algorithm is a useful ingredient in our class. From a coding perspective, this portion of the course has introduced

1. Python's built-in data types,
2. Pandas' DataFrames and simple table manipulations,
3. simple plotting tools via PlotLy, and
4. control-of-flow constructions like loops and "if" statements (today).

You will make extensive use of the basic data types throughout the next 13 weeks, and the DataFrame exposed by Pandas is something you will come back to repeatedly. We will often find ourselves dealing with unstructured data that we will put into a DataFrame to then perform some kind of analysis. PlotLy is, I admit, a stretch. It did get us up and running quickly (and was the only easy tool to let us form dotcharts), but we will see plenty (*plenty*) of other plotting and mapping frameworks throughout the semester in addition to PlotLy.

Today, then, we finish our work on trending. We have created two extra data sets from the DC tweets that should help you experiment with your ideas. First, let's load up the Twitter trends and the tweets from DC during the inauguration weekend.

In [ ]:
from pandas import set_option, read_csv
set_option("display.max_colwidth",140)
set_option("display.max_columns",30)
set_option("display.max_rows",50)

In [ ]:
trends = read_csv('twitter_trending_topics_for_us_120to122_mh2.csv')
trendsdc = trends[(trends["city"]=="Washington")]
trendsdc.shape

In [ ]:
trendsdc.head(5)

We are loading up a slighly different version of the tweets data set. We have added an extra column that is called "epoch". It matches the column in the trends data set above. This way you can refer to epoch 296 and it will refer to the 15 minutes starting at 1:45am on January 23rd (in UTC, 5 hours ahead of NYC time). 

[The data set is here](http://compute-cuj.org/dc_tweets_0120_to_0123_v2_mh2.csv.gz) and will be uploaded to courseworks. Grab it, uncompress it and put it in the same folder as this notebook.

In [ ]:
tweetsdc = read_csv("dc_tweets_0120_to_0123_v2_mh2.csv")
tweetsdc.shape

In [ ]:
tweetsdc.head(5)

To make working with epochs a little easier, we will use the groupby() method we saw last week, to create a simple summary table. For each epoch we will record the time it starts and the number of tweets in that time period. We will want to focus on the time periods with the most activity. 

The code below takes a subset of "tweetsdc" and then groups the rows by "epoch". The ".groups" attributed of the grouped object is a dictionary that has epochs as its keys and then the row numbers of the tweets that belong to the epoch. 

Have a look.

In [ ]:
epochgroups = tweetsdc[["epoch","tweet created_at time","tweet id"]].groupby("epoch")
type(epochgroups.groups)

In [ ]:
epochgroups.groups.keys()

In [ ]:
epochgroups.groups[103]

We can now compute simple functions on each group. This will generate a set of new rows, one for each epoch. While in the past we have called functions like mean() or sum() or count() to summarize the groups of tweets, this time we will create columns that are the result of different computations. For the "epoch start time", we are going to take the time of the first tweet in the epoch group -- we will apply min(). For the "total tweet count", we will want to count all the tweets in each epoch group -- we will use count().

To accomplish this, we will not use .count() or .min(), but instead we will use .agg() for aggregate. It takes a dictionary as its argument where the keys are column names. You can then provide a single function or you can provide another dictionary under this key. If you provide a dictionary, you can specify several new columns that you'd like computed from the old column (the key). 

In agg() we specify the names of the funcions we want applied. 

In [ ]:
epoch = epochgroups.agg({"tweet created_at time":{"epoch start time":"min"},"tweet id":{"total tweet count":"count"}})
epoch.head(5)

So, here we used "tweet created_at time" and made a new column called "epoch start time" by applying the min() function to the group of tweet times for each epoch. Similarly, "tweet id" gave rise to a column named "total tweet count" that simply tallied the number of tweets in each epoch group. 

The result has a multi-index for column names. It also used the epoch as the row index. We don't need the first layer of the column names ("tweet created_at time" and "tweet id" aren't relevant for this table about epochs) and we want to turn the row index into its own column. The two commands below does this.

In [ ]:
epoch.columns = epoch.columns.droplevel()
epoch = epoch.reset_index()
epoch.head(5)

Ha! The .agg() function is a very general expression for what to compute on groups formed by groupby(). I find it pretty expressive. Now, we can take the epoch number and total tweet counts and create a simple bar graph. Dear PlotLy to the rescue.

In [ ]:
from plotly.plotly import iplot, sign_in
import plotly.graph_objs as go

sign_in("","")

mydata = [go.Bar(x=epoch["epoch"],y=epoch["total tweet count"])]
mylayout = go.Layout(autosize=False, width=1000,height=600,margin=go.Margin(l=150,r=50,b=100,t=100,pad=4))
myfigure = go.Figure(data = mydata, layout = mylayout)
iplot(myfigure)

Here are the epochs we might want to focus on. 188 is all about the march, and 296 is a bit more broad it its focus, with a little football coming in.

In [ ]:
epoch.sort_values(by="total tweet count",ascending=False).head(5)

**New Summary Files**

We are now going to look at two files I created to help make your work on the trending algorithms a  little easier. The code to create them is in two appendices below. There is nothing but a concat() and a groupby() there, with an agg() tossed in. But they are all understandable, which is why I included them. You might, for example, want to make similar summaries for entities that I didn't cover.

Download the files from [here](http://compute-cuj.org/hashtagsummary.csv) and [here](http://compute-cuj.org/usersummary.csv").

Let's load them in and have a look. First, we'll look at the hashtag summary.

In [ ]:
hashsummary = read_csv("http://compute-cuj.org/hashtagsummary.csv")
usersummary = read_csv("http://compute-cuj.org/usersummary.csv")

In [ ]:
hashsummary.head(5)

This data set grouped the data by hashtag and epoch. **We turned all the hashtags to lowercase so be careful when you make comparisons**. So each row is a hashtag appearing in an epoch. During that epoch, we record 

1. the number of times the hashtag was tweeted, 
2. how many people tweeted it
3. the largest number of retweets any single tweet with this hashtag received during the epoch,
4. the total number of retweets of the tweets posted during the epoch and containing the hashtag,
5. the largest number of favorites any single tweet with this hashtag received during the epoch,
6. the total number of favorites of the tweets posted during the epoch and containing the hashtag

You also have the epoch number, when it started and the total number of tweets (including those not containing the hashtag) during the epoch. 

For example, "agentsofsheild" was tweeted three times in epoch 85 by a single user. The poor tweet died, receiving no attention or love. These three tweets were three of 1,188 that were posted during epoch 85. Here they are.

In [ ]:
tweetsdc[(tweetsdc["epoch"]==85) & (tweetsdc["hashtag #1"].str.contains("agentsofshield",case=False))]

Try looking Demetre93122995 up... Bot? Human?

The second summary data set begins with the tweetsdc data as well, this time grouping by epoch and screen_name. Below we see 2GOOLIVEE tweeted 4 times in epoch 85. These tweets were retweeted once and not favorited. The screen_name's were not set to lowercase as I don't believe Twitter changes the case when they publish tweets.

In [ ]:
usersummary.head(5)

In [ ]:
tweetsdc[(tweetsdc["epoch"]==85) & (tweetsdc["user's screen_name"] == "2GOOLIVEE")]

In [ ]:
tweetsdc[tweetsdc["user's screen_name"] == "2GOOLIVEE"]

Now, use these to investigate the algorithms you have planned. If you want other data in the summaries, let us know. Take 45 minutes to really look at the data and the kinds of decisions you might make.

**Designing an Trend Algorithm**

We are now going to give you some basic code stubs to program your algorithm. We will need two ingredients. The first is a construction called a "function". we have been copy and pasting code when we wanted to repeat a computation, slightly altered. The formal mechanism for this is to corral the instructions into a function that you can execute, varying arguments. 

Below we define a function that takes two arguments. It is called "trend" and it needs a value of "e", the epoch, and "n" the number of items you'd like in the trend list. So trend(188,10) would give you a list of 10 items for epoch 188. Get it? The "n=5" in the argument list means that if you don't provide a second argument, the function will assume you meant 5. Because "e" doesn't have a default value, you have to set one. 

The function looks at the hashsummary and pulls out just the data for epoch "e". It then makes a copy of the data. The thing about Python functions like this is that you can end up making changes to hashsummary, something we want to avoid. So we will copy the portion of the dataframe we want. 

From there, we just sort according to tweet count and keep the top "n" hashtags. This means the first entry received the ost tweets, the second, the second most, and so on. We finish by "return"ing the result. We can then do with that what we will. 

So, we execute the code to create the function. We can check the type of "trend" to see that it is, indeed, a function. And then we try it out!

In [ ]:
def trend(e,n=5):
    
    # sort the hashtags in the epoch by the number of tweets they received
    hashtags = hashsummary[hashsummary["epoch"]==e].copy()

    # sort the hashtags in the epoch by the number of tweets they received
    hashtags = hashtags.sort_values("tweet count",ascending=False)["hashtag"]
    
    # turn the series into a simple list
    top = list(hashtags[:n])
    
    # return the top n trends!
    return top

In [ ]:
type(trend)

In [ ]:
trend(282,10)

Exercise the code above a few times. Try it with different epochs and see how the top items change. You can also compare them to the trends in that epoch as computed by Twitter.

In [ ]:
trendsdc[trendsdc["epoch"]==282]

Twitter has a fair number people and organizations that are not hash tags, but are pulled from the text of the tweets. We will create a summary of these for you for your drill. Still, notice how the simpleminded algorithm isn't horrible. There are sports-related topics and some having to do with the after-march.

Now, suppose we want to see how the top 5 vary over time. Perhaps we want to compute the trends for a range of epochs. Below we use our second new construction of the day, a for-loop. These loops have a simple syntax. They begin with the word "for" and then name a variable to loop over. The set of values we traverse follow the "in". In this case, it's a list of integer values from 200 to 219.

In [ ]:
range(200,220)

In [ ]:
for e in range(200,220):
    print trend(e)

The code above updates the value of e with successive values from the list of integers. At first we have e=200 and we compute trend(200). Then e=201 and we compute trend(201). Get the idea? Each pass of the loop just prints out the top 5 trends. 

We can also catch each list, storing them in a list. Remember our list-of-lists? We can store them and then load them into a DataFrame! How's this? Each pass of the loop appends the trend() output to a growing list called "topics". We then turn topics into a DataFrame.

In [ ]:
from pandas import DataFrame, Series

In [ ]:
topics = []

for e in range(280,300):
    topics.append(trend(e))
    
DataFrame(topics)

Here we compile the output into one big trend list and then covert it to a Series rather than a DataFrame. This way we can call value_counts() and see how many times a topic has been in the top 5.

In [ ]:
topics = []

for e in range(290,300):
    topics = topics+trend(e)
    
Series(topics).value_counts()

OK so the hashtag with the most tweets is pretty easy. Let's change it and try something else. Just a single column for now, we'll get more expressive later.

In [ ]:
# put your code here defining a new function trend



And run it!

In [ ]:
topics = []

for e in range(290,300):
    topics.append(trend(e))
    
DataFrame(topics)

In [ ]:
def trend(e,n=5):
    
    # sort the hashtags in the epoch by the number of tweets they received
    hashtags = hashsummary[hashsummary["epoch"]==e].copy()
    
    # sort the hashtags in the epoch by the number of tweets they received
    hashtags["score"] = hashtags["tweet count"]+hashtags["user count"]
    
    hashtags = hashtags.sort_values("score",ascending=False)["hashtag"]
    top = list(hashtags[:n])
    
    return top

In [ ]:
topics = []

for e in range(290,300):
    topics.append(trend(e))
    
DataFrame(topics)

Below we expand things a little. Suppose we want to involve the history of how often a hashtag appears. We can do this with a loop, calling trend() just as we have before. This time, we can do the call inside a new function which we will name trendhist(). It builds up a Series of old top-n topics and then filters the current trending topics, keeping those that have not appeared too often.

In [ ]:
def trendhist(epoch,n=5):
    
    # collect the old topics into a series
    
    oldtopics = []
    
    for e in range(epoch-5,epoch):
        oldtopics = oldtopics + trend(e)
    
    oldtopics = Series(oldtopics).value_counts()
    
    # now pull the new topics
    
    newtopics = trend(epoch,20)

    # and test if they appeared too many times in the past.
    # here we ask that they appeared in the top 5 less than 3 days,
    # otherwise we leave them out
    
    final = []
    
    print "--"*5
    
    for t in newtopics:
        
        if (not t in oldtopics) or (oldtopics[t] < 3):
            final.append(t)
        else:
            print "Dropping", t
            
    return final[:5]

In [ ]:
trendhist(290)

In [ ]:
topics = []

for e in range(290,300):
    topics.append(trendhist(e))
    
DataFrame(topics)

Now it's your turn! Cook up something and see what happens!

In [ ]:
# your code here



**Appendix: Making the Hashtag Summary**

Here we go briefly over how to make the hashsummary DataFrame from the tweetsdc data. As we had done in a previous lecture, we create 3 Dataframes, one for each of "hashtag \#1", "hashtag \#2" and "hashtag \#3". We rename the columns so each one containing hashtags  is called just "hashtag". 

We then use concat() to stack them ontop of each other to make a big data frame. We remove the rows with NaN values (those for which there wasn't a hashtag in the tweet) and turn everything lowercase. Whew!

In [ ]:
from pandas import concat

# Stack all 3 data sets using concat()
hashtags1 = tweetsdc[["epoch","tweet created_at time","user's screen_name","retweet count","favorite count","hashtag #1"]]
hashtags1 = hashtags1.rename(columns={"hashtag #1":"hashtag"})

hashtags2 = tweetsdc[["epoch","tweet created_at time","user's screen_name","retweet count", "favorite count","hashtag #2"]]
hashtags2 = hashtags2.rename(columns={"hashtag #2":"hashtag"})

hashtags3 = tweetsdc[["epoch","tweet created_at time","user's screen_name","retweet count", "favorite count","hashtag #3"]]
hashtags3 = hashtags3.rename(columns={"hashtag #3":"hashtag"})

hashtags = concat([hashtags1,hashtags2,hashtags3])

# remove any nan entries, indicating tweets without hashtags
hashtags = hashtags[~hashtags["hashtag"].isnull()]

# make hashtags lowercase
hashtags["hashtag"] = hashtags["hashtag"].str.lower()

hashtags.shape

In [ ]:
hashtags.head(5)

Form groups having unique epoch-hashtag pairs. We then create new columns that sum up the retweets and favorites, pull out the number of retweets earned by a single tweet. Ditto for favorites. And look at how many unique users there are. Compare the agg() function here to the one above when we made "epoch".

In [ ]:
hashgroups = hashtags.groupby(["epoch","hashtag"])

In [ ]:
hashsummary = hashgroups.agg({"retweet count":{"total retweets":"sum","max retweets":"max"},"favorite count":{"total favorites":"sum","max favorites":"max"},"user's screen_name":{"tweet count":"count","user count":"nunique"}})
hashsummary.head(5)

Again, drop the extra level of column names and turn the (multi-)index into rows.

In [ ]:
hashsummary.columns=hashsummary.columns.droplevel()
hashsummary = hashsummary.reset_index()

In [ ]:
hashsummary.head()

**Appendix: "merging" the Hashtag Summary with the Epoch Table**

Now, we want to take the data from each epoch in our "epoch" table and add it to the hashsummary. This way we have the extra references of the time the epoch began and the total number of tweets during that epoch. The operation here uses a command called merge() that will add data from one data set to another. We will use the column epoch as the "key". That means when we see epoch 85 in the hashsummary, we will add the row of data corresponding to epoch 85 from the epoch table.

We will have a lot more to say about this later in the term. But for now, notice that we can easily expand a data set by adding new details.

In [ ]:
from pandas import merge

hashsummary = merge(hashsummary,epoch,on="epoch")
hashsummary.head()

In [ ]:
hashsummary.sort_values(["tweet count"],ascending=False).head(5)

In [ ]:
hashsummary[hashsummary["epoch"]==298].sort_values(by="tweet count",ascending=False).head(5)

In [ ]:
hashsummary.to_csv("hashtagsummary.csv",index=False)

**Appendix: Making the User Table**

Finally, making the user table. Here we again form a dataframe (no concat() needed here) from the original tweets data. We then groupby() epoch and the user's screen name. The agg() should be old news now as we form variables from the retweet count, the favorite count, and tally the total number of tweets from this user during the epoch.

We drop a row of column names and reset the index and we're done!

In [ ]:
from pandas import concat

users = tweetsdc[["epoch","tweet created_at time","user's screen_name","retweet count","favorite count"]]
users.shape

In [ ]:
usergroups = users.groupby(["epoch","user's screen_name"])
usersummary = usergroups.agg({"retweet count":{"total retweets":"sum"},"favorite count":{"total favorite":"sum"},"tweet created_at time":{"total tweets":"count"}})
usersummary.head()

In [ ]:
usersummary.columns = usersummary.columns.droplevel()
usersummary = usersummary.reset_index()
usersummary.head()

In [ ]:
usersummary.sort_values(by="total retweets",ascending=False).head(5)

In [ ]:
usersummary.to_csv("usersummary.csv",index=False)